<a href="https://colab.research.google.com/github/yalem123/Fraud-detection-project-Adey-Innovations/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Take a smaller sample for safe processing
fraud_small = fraud_df.sample(n=10000, random_state=42).reset_index(drop=True)

# Prepare features and target
X = fraud_small.drop(columns=['class', 'signup_time', 'purchase_time', 'ip_address', 'ip_int'])
y = fraud_small['class']

# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check class distribution
print("Before SMOTE:\n", y_train.value_counts())
print("\nAfter SMOTE:\n", y_train_resampled.value_counts())


In [ ]:
import pandas as pd
import ipaddress

# Upload files again
from google.colab import files
uploaded = files.upload()

# Load datasets
fraud_df = pd.read_csv('Fraud_Data.csv')
ip_df = pd.read_csv('IpAddress_to_Country.csv')

# Sample 30% to reduce memory usage
fraud_df = fraud_df.sample(frac=0.3, random_state=42).reset_index(drop=True)

# Convert timestamps
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])

# Convert IPs to integers
fraud_df['ip_int'] = fraud_df['ip_address'].astype(int)
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

# Match IP to country
ip_df = ip_df.sort_values(by='lower_bound_ip_address')
def find_country(ip_int):
    match = ip_df[(ip_df['lower_bound_ip_address'] <= ip_int) & (ip_df['upper_bound_ip_address'] >= ip_int)]
    return match.iloc[0]['country'] if not match.empty else 'Unknown'
fraud_df['country'] = fraud_df['ip_int'].apply(find_country)

# Feature engineering
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek
fraud_df['time_since_signup'] = (fraud_df['purchase_time'] - fraud_df['signup_time']).dt.total_seconds()

# Preview
fraud_df[['purchase_time', 'hour_of_day', 'day_of_week', 'time_since_signup', 'country']].head()


Saving Fraud_Data.csv to Fraud_Data (1).csv
Saving IpAddress_to_Country.csv to IpAddress_to_Country (1).csv


,purchase_time,hour_of_day,day_of_week,time_since_signup,country
0,2015-03-04 01:26:04,1,2,3425424.0,Morocco
1,2015-06-25 12:00:50,12,3,65912.0,Korea Republic of
2,2015-07-11 07:35:20,7,5,8475462.0,United States
3,2015-10-17 18:00:17,18,5,5259328.0,United States
4,2015-03-21 14:23:17,14,5,4168527.0,Germany


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Take a smaller sample for safe processing
fraud_small = fraud_df.sample(n=10000, random_state=42).reset_index(drop=True)

# Prepare features and target
X = fraud_small.drop(columns=['class', 'signup_time', 'purchase_time', 'ip_address', 'ip_int'])
y = fraud_small['class']

# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check class distribution
print("Before SMOTE:\n", y_train.value_counts())
print("\nAfter SMOTE:\n", y_train_resampled.value_counts())


Before SMOTE:
 class
0    7262
1     738
Name: count, dtype: int64

After SMOTE:
 class
0    7262
1    7262
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit and transform training data
X_train_scaled = scaler.fit_transform(X_train_resampled)

# Transform test data (note: use original test set, not resampled)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve

# --- Logistic Regression ---
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train_resampled)
y_pred_lr = lr.predict(X_test_scaled)

print("Logistic Regression Results:")
print(classification_report(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

# --- XGBoost ---
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb.predict(X_test)

print("\nXGBoost Results:")
print(classification_report(y_test, y_pred_xgb))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))


Logistic Regression Results:
              precision    recall  f1-score   support

           0       0.95      0.66      0.78      1816
           1       0.16      0.66      0.26       184

    accuracy                           0.66      2000
   macro avg       0.56      0.66      0.52      2000
weighted avg       0.88      0.66      0.73      2000

Confusion Matrix:
 [[1190  626]
 [  63  121]]


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [08:36:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost Results:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1816
           1       0.92      0.57      0.70       184

    accuracy                           0.96      2000
   macro avg       0.94      0.78      0.84      2000
weighted avg       0.95      0.96      0.95      2000

Confusion Matrix:
 [[1807    9]
 [  80  104]]
